In [16]:
!pwd

/c/Users/user/Desktop/ML_Marathon/EDA


In [17]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 3E41-B1A3

 Directory of c:\Users\user\Desktop\ML_Marathon\EDA

29/01/2024  19:03    <DIR>          .
29/01/2024  19:03    <DIR>          ..
29/01/2024  19:03    <DIR>          EDA
29/01/2024  19:16           133,500 titanic_EDA.ipynb
               1 File(s)        133,500 bytes
               3 Dir(s)   7,624,003,584 bytes free


In [1]:
import pandas as pd
import numpy as np
import sklearn
import xgboost as xgb

In [33]:
titanic_train = pd.read_csv(r'C:\Users\user\Desktop\ML_Marathon\Data\Datasets\spaceship-titanic\train.csv')
titanic_test = pd.read_csv(r'C:\Users\user\Desktop\ML_Marathon\Data\Datasets\spaceship-titanic\test.csv')
titanic_train.info()
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  

In [3]:
titanic_test.head()
titanic_train.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


### Drop the colummns [names,id,cabin column]

In [4]:
titanic_test.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'],
      dtype='object')

In [5]:
titanic_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [23]:
titanic_test = titanic_test.drop(['PassengerId','Name','Cabin'],axis = 1)
titanic_train = titanic_train.drop(['PassengerId','Name','Cabin'],axis=1)

In [4]:
titanic_test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0
2,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0
3,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0
4,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0


### Label Encoding [VIP,Destination,CryoSleep]

In [5]:
from sklearn import *

In [24]:
# Missing Values
titanic_train = titanic_train.fillna({'Age': titanic_train['Age'].mean(), 'RoomService': titanic_train['RoomService'].median(),'ShoppingMall':0,'Spa':0,'VRDeck':0,'FoodCourt':0})
titanic_test = titanic_test.fillna({'Age': titanic_test['Age'].mean(), 'RoomService': titanic_test['RoomService'].median(),'ShoppingMall':0,'Spa':0,'VRDeck':0,'FoodCourt':0})

titanic_train = titanic_train.fillna({'HomePlanet': 'Earth', 'CryoSleep': False, 'Destination': 'TRAPPIST-1e', 'VIP': False})
titanic_test = titanic_test.fillna({'HomePlanet': 'Earth', 'CryoSleep': False, 'Destination': 'TRAPPIST-1e', 'VIP': False})

In [25]:
lb = preprocessing.LabelBinarizer()
le = preprocessing.LabelEncoder()
#lb.fit(titanic_train.VIP) # true or false values
dest_tran = le.fit(titanic_train['Destination'])
planet_tran = le.fit(titanic_train['HomePlanet'])

In [26]:
cryo_tran = preprocessing.LabelEncoder()
dest_tran = preprocessing.LabelEncoder()
# Train file
titanic_train['HomePlanet'] = planet_tran.fit_transform(titanic_train['HomePlanet'])
titanic_train['Destination']  = dest_tran.fit_transform(titanic_train['Destination'])
titanic_train['CryoSleep'] = cryo_tran.fit_transform(titanic_train['CryoSleep'])
titanic_train['VIP'] = cryo_tran.fit_transform(titanic_train['VIP'])
# Test file
titanic_test['CryoSleep'] = cryo_tran.fit_transform(titanic_test['CryoSleep'])
titanic_test['Destination']  = dest_tran.fit_transform(titanic_test['Destination'])
titanic_test['HomePlanet'] = planet_tran.fit_transform(titanic_test['HomePlanet'])
titanic_test['VIP'] = cryo_tran.fit_transform(titanic_test['VIP'])


In [9]:
# Encoded Labels
print(dest_tran.classes_,planet_tran.classes_,cryo_tran.classes_)

['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e'] ['Earth' 'Europa' 'Mars'] [False  True]


In [14]:
titanic_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True


In [27]:
titanic_test.head()


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0,1,2,27.0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,2,19.0,0,0.0,9.0,0.0,2823.0,0.0
2,1,1,0,31.0,0,0.0,0.0,0.0,0.0,0.0
3,1,0,2,38.0,0,0.0,6652.0,0.0,181.0,585.0
4,0,0,2,20.0,0,10.0,0.0,635.0,0.0,0.0


In [12]:
sk_model = tree.DecisionTreeClassifier()
xgb_model = xgb.XGBClassifier()

x_train = titanic_train.iloc[:,:-1]
y_train = titanic_train.iloc[:,-1]
#acc = metrics.accuracy_score()

In [13]:
x_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0


In [15]:
sk_model.fit(x_train,y_train)

DecisionTreeClassifier()

In [ ]:
x_train.head()


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0


In [14]:
titanic_test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0,1,2,27.0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,2,19.0,0,0.0,9.0,0.0,2823.0,0.0
2,1,1,0,31.0,0,0.0,0.0,0.0,0.0,0.0
3,1,0,2,38.0,0,0.0,6652.0,0.0,181.0,585.0
4,0,0,2,20.0,0,10.0,0.0,635.0,0.0,0.0


In [17]:
predict = sk_model.predict(titanic_test)
predict

array([ True, False,  True, ...,  True,  True, False])

In [34]:
Test_PassengerId = titanic_test.PassengerId


In [20]:
print(len(Test_PassengerId)==len(predict))

True


In [21]:
data = pd.Series(predict,index=Test_PassengerId)
data.head()

PassengerId
0013_01     True
0018_01    False
0019_01     True
0021_01     True
0023_01    False
dtype: bool

In [22]:
xgb_model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [28]:
xgb_predict = xgb_model.predict(titanic_test)

In [29]:
acc = metrics.accuracy_score(predict,xgb_predict)

In [30]:
acc

0.8569090483984101

In [31]:
i_acc = metrics.accuracy_score(xgb_predict,predict)

In [35]:
xgb_data = pd.Series(xgb_predict,index =Test_PassengerId)

In [37]:
xgb_data.head()

PassengerId
0013_01    1
0018_01    0
0019_01    1
0021_01    1
0023_01    1
dtype: int32

In [38]:
x = []
for i in xgb_predict:
    if i == 1:
        x.append('true')
    else:
        x.append('false')
        
x

['true',
 'false',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'false',
 'false',
 'true',
 'true',
 'false',
 'false',
 'false',
 'true',
 'true',
 'false',
 'true',
 'false',
 'true',
 'true',
 'false',
 'false',
 'true',
 'true',
 'false',
 'true',
 'true',
 'false',
 'true',
 'true',
 'false',
 'false',
 'true',
 'true',
 'false',
 'false',
 'true',
 'false',
 'true',
 'true',
 'false',
 'true',
 'false',
 'false',
 'true',
 'false',
 'true',
 'true',
 'false',
 'true',
 'true',
 'true',
 'false',
 'true',
 'true',
 'false',
 'true',
 'true',
 'false',
 'true',
 'true',
 'false',
 'true',
 'false',
 'false',
 'false',
 'false',
 'true',
 'false',
 'false',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'true',
 'false',
 'false',
 'false',
 'false',
 'true',
 'true',
 'false',
 'true',
 'true',
 'false',
 'false',
 'false',
 'false',
 'false',
 'false',
 'true',
 'false',
 'true',
 'false',
 'true',
 'true',
 'true',
 'false',
 'true',
 'false',

In [40]:
xgb_data = pd.Series(x,index =Test_PassengerId)
xgb_data.head()

PassengerId
0013_01     true
0018_01    false
0019_01     true
0021_01     true
0023_01     true
dtype: object